In [ ]:
#importing libraries
import requests # enables api calls
import pandas as pd
import time

extarcting video metrics on the youtube channel of neo4j

In [ ]:
#Keys
API_KEY = "xxxx" #youtube api
CHANNEL_ID = "UCvze3hU6OZBkB1vkhH2lH9Q" #neo4j channel ID

In [ ]:
def get_video_details(video_id):

    #collecting view, like, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count,comment_count


In [ ]:
def get_videos(df):
    pageToken = ""
    import warnings

    while True:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        try:
            response = requests.get(url).json()
            time.sleep(1)  # Give it a second before starting the for loop

            for video in response['items']:
                if video['id']['kind'] == "youtube#video":
                    video_id = video['id']['videoId']
                    video_title = video['snippet']['title']
                    video_title = str(video_title).replace("&", "")
                    upload_date = video['snippet']['publishedAt']
                    upload_date = str(upload_date).split("T")[0]

                    # Retrieve video details efficiently without making additional API requests
                    video_details = get_video_details(video_id)
                    if video_details is not None:
                        view_count, like_count, comment_count = video_details
                        with warnings.catch_warnings():  # Use a context manager to catch and ignore the warning
                          warnings.simplefilter("ignore")
                        # Create a DataFrame for each video's details
                          video_df = pd.DataFrame({'video_id': [video_id], 'video_title': [video_title],
                                                "upload_date": [upload_date], "view_count": [view_count],
                                                "like_count": [like_count], "comment_count": [comment_count]})

                          df=df.append(video_df,ignore_index=True)  # Append the DataFrame to the list

            if 'nextPageToken' in response:
                pageToken = "pageToken=" + response['nextPageToken']
            else:
                break  # No more pages

        except requests.exceptions.RequestException as e:
            print(f"An error occurred during the API request: {str(e)}")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            break

    return df

In [ ]:
#main

#build our dataframe
df2 = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"])

df2 = get_videos(df2)


An error occurred: 'likeCount'


In [ ]:
df2



,video_id,video_title,upload_date,view_count,like_count,comment_count
0,Tx1zCnlc0_g,"Going Meta - A Series on Graphs, Semantics and...",2023-09-06,0,0,0
1,u0yEI-xU2t4,APAC Training Series - Knowledge Graphs with C...,2023-06-14,2292,55,2
2,mMkXnmueasA,Neo4j and Generative AI,2023-06-13,5038,57,6
3,As2UBg56b3o,Neo4j Live: GraphGPT,2023-05-10,3015,55,1
4,NUuI9NkOfZo,Neo4j Workspace — New Updates for Import and Q...,2023-05-05,647,11,0
...,...,...,...,...,...,...
60,XJh4mneRRhY,NOM Bytes 3 - Running like a dream. Review the...,2022-06-28,302,3,0
61,Tz727w5_wRo,NOM Bytes 1. Are we good? Get an estate-wide v...,2022-06-28,632,9,2
62,nnQFJ6wfnpk,Neo4j Live: Improve crime investigations with ...,2022-06-23,906,21,0
63,KCQTH-EwO_c,Neo4j Under the Hood - Conclusion - The Future...,2022-06-15,1134,16,0


In [ ]:
#storing the data in csv file
df2.to_csv('youtube_vids.csv')